<a href="https://colab.research.google.com/github/ttury/Deep-Learning-For-Natural-Language-Processing/blob/master/Text_Generation_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('./drive/MyDrive/datasets/archive/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [8]:
print('열의 개수: ', len(df.columns))
print(df.columns)
print(df['headline'].isnull().values.any())
# isnull 함수는 결측값이 있으면 True, 없으면 False를 반환
# any 함수는 True 값이 하나라도 있으면 True, 없으면 False를 반환

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')
False


In [13]:
headline = list(df.headline.values)
print(headline[:5])
print('총 샘플의 개수 : {}'.format(len(headline)))

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell', 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.', 'The New Noma, Explained', 'Unknown', 'Unknown']
총 샘플의 개수 : 1324


In [17]:
headline = [corpus for corpus in headline if corpus != "Unknown"]
print(headline[:5])
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell', 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.', 'The New Noma, Explained', 'How a Bag of Texas Dirt  Became a Times Tradition', 'Is School a Place for Self-Expression?']
노이즈값 제거 후 샘플의 개수 : 1214


In [21]:
def repreprocessing(sample):
  sample = sample.encode('utf8').decode('ascii', 'ignore') # 아스키 코드만 남기고 삭제
  return ''.join(ch for ch in sample if ch not in punctuation).lower() # 구두점 제거, 소문자 변환

text = [repreprocessing(sample) for sample in headline]
print(text[:5])

['former nfl cheerleaders settlement offer 1 and a meeting with goodell', 'epa to unveil a new rule its effect less science in policymaking', 'the new noma explained', 'how a bag of texas dirt  became a times tradition', 'is school a place for selfexpression']


In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩은 인덱스가 0부터 시작하기 때문에
# 인덱스를 1부터 시작하기 위해 단어 집합의 크기보다 1 크게 지정

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [44]:
sequences = list()

for line in text:
  encoded = tokenizer.texts_to_sequences([line])[0] # 정수 인코딩
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [45]:
index_to_word = dict()
for key, value in tokenizer.word_index.items(): # dictionary key, value swap
  index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [46]:
max_len = max(len(sample) for sample in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

샘플의 최대 길이 : 24
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [47]:
sequences = np.array(sequences)
x = sequences[:, :-1]
y = sequences[:, -1]

print(x.shape)
print(x[:3])
print(y[:3])

(7803, 23)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115]


In [48]:
y = to_categorical(y, num_classes=vocab_size)
print(y[0])

[0. 0. 0. ... 0. 0. 0.]


In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128)) # hidden_size = 128
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 9s - loss: 7.6500 - accuracy: 0.0270
Epoch 2/200
244/244 - 7s - loss: 7.1167 - accuracy: 0.0306
Epoch 3/200
244/244 - 7s - loss: 6.9772 - accuracy: 0.0361
Epoch 4/200
244/244 - 7s - loss: 6.8436 - accuracy: 0.0408
Epoch 5/200
244/244 - 7s - loss: 6.6912 - accuracy: 0.0454
Epoch 6/200
244/244 - 7s - loss: 6.5238 - accuracy: 0.0486
Epoch 7/200
244/244 - 7s - loss: 6.3428 - accuracy: 0.0523
Epoch 8/200
244/244 - 7s - loss: 6.1483 - accuracy: 0.0566
Epoch 9/200
244/244 - 7s - loss: 5.9522 - accuracy: 0.0607
Epoch 10/200
244/244 - 7s - loss: 5.7631 - accuracy: 0.0670
Epoch 11/200
244/244 - 7s - loss: 5.5813 - accuracy: 0.0718
Epoch 12/200
244/244 - 7s - loss: 5.4098 - accuracy: 0.0773
Epoch 13/200
244/244 - 7s - loss: 5.2498 - accuracy: 0.0825
Epoch 14/200
244/244 - 7s - loss: 5.0997 - accuracy: 0.0898
Epoch 15/200
244/244 - 7s - loss: 4.9574 - accuracy: 0.0984
Epoch 16/200
244/244 - 7s - loss: 4.8183 - accuracy: 0.1066
Epoch 17/200
244/244 - 7s - loss: 4.6868 - accura

In [50]:
def generate_sent(model, tokenizer, current_word, n):
  sentence = current_word
  for _ in range(n):
    encoded = tokenizer.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=23, padding='pre')
    result = model.predict_classes(encoded, verbose=0)
    for word, index in tokenizer.word_index.items():
      if index == result:
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
        break
  return sentence

In [59]:
input_word = input()
n = int(input())

print(generate_sent(model, tokenizer, input_word, n))

how
10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


how to make a crossword puzzle epa by political king science


In [60]:
model.save('./drive/MyDrive/models/LSTM_toy')
# new_model = tf.keras.models.load_model('./drive/MyDrive/models/LSTM_toy') 로 모델 불러오기 가능

INFO:tensorflow:Assets written to: ./drive/MyDrive/models/LSTM_toy/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/models/LSTM_toy/assets


In [65]:
import pickle

with open('./drive/MyDrive/models/LSTM_toy_tokenizer', 'wb') as f:
	pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

'''
with open('./drive/MyDrive/models/LSTM_toy_tokenizer', 'rb') as f:
	tokenizer = pickle.load(f)

위의 코드로 tokenzier 객체 불러오기 가능
'''